## loading word embeddings

In [0]:
def download_from_google_drive(FILEID, FILENAME):
    # found on https://medium.com/@paudelanjanchandra/download-google-drive-files-using-wget-3c2c025a8b99
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={FILEID}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={FILEID}" -O {FILENAME} && rm -rf /tmp/cookies.txt


In [0]:
import sklearn
import numpy as np
import pandas as pd
from  sklearn.metrics.pairwise import cosine_similarity

import heapq
import nltk
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# downloading in.npz
# https://drive.google.com/file/d/1sl0Csw3uY0tkfNaMYAcbWKpedffLdz4G/view?usp=sharing
download_from_google_drive("1sl0Csw3uY0tkfNaMYAcbWKpedffLdz4G", "in.npz")


--2019-12-15 10:51:30--  https://docs.google.com/uc?export=download&confirm=bs50&id=1sl0Csw3uY0tkfNaMYAcbWKpedffLdz4G
Resolving docs.google.com (docs.google.com)... 172.217.204.100, 172.217.204.139, 172.217.204.113, ...
Connecting to docs.google.com (docs.google.com)|172.217.204.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-7g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7aee059ooo0safs3ndgaij4fk7ar065d/1576404000000/16726935551075372216/*/1sl0Csw3uY0tkfNaMYAcbWKpedffLdz4G?e=download [following]
--2019-12-15 10:51:30--  https://doc-04-7g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7aee059ooo0safs3ndgaij4fk7ar065d/1576404000000/16726935551075372216/*/1sl0Csw3uY0tkfNaMYAcbWKpedffLdz4G?e=download
Resolving doc-04-7g-docs.googleusercontent.com (doc-04-7g-docs.googleusercontent.com)... 74.125.31.132, 2607:f8b0:400c:c02::84
Connecting to doc-04-7g-docs.googleusercontent.

In [0]:
# downloading out.npz
# https://drive.google.com/file/d/1dY_ErHuTvjzIPfT4kNyTCQTC1bYuJF7h/view?usp=sharing
download_from_google_drive("1dY_ErHuTvjzIPfT4kNyTCQTC1bYuJF7h", "out.npz")


--2019-12-15 10:51:51--  https://docs.google.com/uc?export=download&confirm=u0Ed&id=1dY_ErHuTvjzIPfT4kNyTCQTC1bYuJF7h
Resolving docs.google.com (docs.google.com)... 74.125.31.113, 74.125.31.101, 74.125.31.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.31.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-7g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/e4n7922g5nq8me8jgttd0cinem464b4o/1576404000000/16726935551075372216/*/1dY_ErHuTvjzIPfT4kNyTCQTC1bYuJF7h?e=download [following]
--2019-12-15 10:51:51--  https://doc-00-7g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/e4n7922g5nq8me8jgttd0cinem464b4o/1576404000000/16726935551075372216/*/1dY_ErHuTvjzIPfT4kNyTCQTC1bYuJF7h?e=download
Resolving doc-00-7g-docs.googleusercontent.com (doc-00-7g-docs.googleusercontent.com)... 74.125.31.132, 2607:f8b0:400c:c02::84
Connecting to doc-00-7g-docs.googleusercontent.com (doc

In [0]:
in_npz = np.load('in.npz')
in_words = in_npz['words']
in_vectors = in_npz['vectors']
in_npz.close()


In [0]:
out_npz = np.load('out.npz')
out_words = out_npz['words']
out_vectors = out_npz['vectors']
out_npz.close()

## test and compare to paper

In [0]:
print(np.where(in_words == "yale")[0][0])

out_vector = out_vectors[np.where(out_words == "yale")[0][0]]
print(out_words[np.where(out_words == "yale")[0][0]])
in_vector = in_vectors[np.where(out_words == "yale")[0][0]]
print(in_words[np.where(out_words == "yale")[0][0]])


2710094
yale
yale


In [0]:

in_out = cosine_similarity(out_vectors,in_vector.reshape(1,-1))
in_in = cosine_similarity(in_vectors,in_vector.reshape(1,-1))
out_out = cosine_similarity(out_vectors,out_vector.reshape(1,-1))
out_in = cosine_similarity(in_vectors,out_vector.reshape(1,-1))


In [0]:
print(np.amax(in_out))
print(np.where(in_out == np.amax(in_out))[0][0])
print(out_words[np.where(in_out == np.amax(in_out))[0][0]])

0.28634506
875351
eminem


In [0]:

in_out_r = np.asarray(heapq.nlargest(15, range(len(in_out)), in_out.take))
in_in_r = np.asarray(heapq.nlargest(15, range(len(in_in)), in_in.take))
out_out_r = np.asarray(heapq.nlargest(15, range(len(out_out)), out_out.take))
out_in_r = np.asarray(heapq.nlargest(15, range(len(in_out)), out_in.take))

In [0]:
for i in in_out_r, in_in_r, out_out_r, out_in_r:
  print(out_words[i])

['yale' 'glp050' 'faculty' 'sznol' 'alumni' '5400ln' 'preregistration'
 'orientation' 'glc050' '1217a' 'haven' 'classesv2' 'graduate' 'mba'
 'mpb040']
['yale' 'harvard' 'nyu' 'cornell' 'quinnipiac' 'tulane' 'tufts' 'uconn'
 'ucsd' 'vanderbilt' 'emory' 'uic' 'ucla' 'gwu' 'hamline']
['yale' 'uconn' 'harvard' 'tulane' 'nyu' 'tufts' 'uic' 'cornell' 'samford'
 'njit' 'purdue' 'drexel' 'vanderbilt' 'unm' 'csuf']
['endante' '2eamedd' 'ncccbanner'
 'qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq' 'jytr'
 'listomania' 'hdcentral' 'santabarbaraindependent' 'vozme' 'ollie1'
 'aibexpress' 'cvbd' 'mysnc' 'gooddining' 'dermtech']


## query expansion

In [0]:

data = pd.read_csv("/content/drive/My Drive/2018CommonCoreTrack/topics.csv") 
data.head()


,Unnamed: 0,num,title,desc,narr
0,0,Number: 321,\nWomen in Parliaments \n,Description:\nPertinent documents will reflec...,Narrative\nPertinent documents relating to th...
1,1,Number: 336,\nBlack Bear Attacks \n,Description:\nA relevant document would discu...,Narrative\nIt has been reported that food or ...
2,2,Number: 341,\nAirport Security \n,Description:\nA relevant document would discu...,Narrative\nA relevant document would contain ...
3,3,Number: 347,\nWildlife Extinction \n,Description:\nThe spotted owl episode in Amer...,Narrative\nA relevant item will specify the c...
4,4,Number: 350,\nHealth and Computer Terminals \n,Description:\nIs it hazardous to the health o...,Narrative\nRelevant documents would contain a...


creating expanded queries

In [0]:

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

expaned_query = []
for query in data["title"][:20]:
    wordss = [w for w in nltk.word_tokenize(query) if w not in stop_words and w != ',' and  w != "' '"]
    av_vector = np.zeros(len(out_vectors[0]))
    for j in wordss:
        av_vector = av_vector + out_vectors[np.where(in_words == j.lower())[0][0]]
    in_out = cosine_similarity(in_vectors, (sklearn.preprocessing.normalize(av_vector.reshape(1,-1))/len(wordss)).reshape(1,-1))
    in_out_r2 = np.asarray(heapq.nlargest(15, range(len(in_out)), in_out.take))
    query_terms = [out_words[i] for i in in_out_r2]
    expaned_query.append(query_terms)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
for i in range(20):
    print(data["title"][i])
    print(expaned_query[i])


Women in Parliaments 

['qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq', 'ebayx', 'cvbd', 'listomania', 'vozme', 'placebook', 'jantovan', 'thechalkboardmag', 'extremenudefakes', 'factsandchicks', 'sinobiological', 'talcosis', 'maxanalporn', 'infonatel', 'tvuzivo']

Black Bear Attacks 

['qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq', 'vozme', 'listomania', 'cvbd', 'sinobiological', 'bearingarms', 'maxanalporn', 'placebook', 'ebayx', 'yahocomo', 'fldinst', 'sexwurst', 'yeppudaa', 'talcosis', 'infonatel']

Airport Security 

['qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq', 'cvbd', 'vozme', 'listomania', 'netthe', 'infonatel', 'maxanalporn', 'atsid', 'pahcs', 'talcosis', 'acquience', 'presafe', 'fearthefin', 'maturegaystories', '0bing']

Wildlife Extinction 

['qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq', 'vozme', 'cvbd', 'listomania', 'sinobiological', 'fldinst', 'placebook', 'bearingarms', 'goodnewsnetwork', 'thechalkboardmag', 'bblo

In [0]:
expanded_list_in_out = [data["title"][0]. replace(" \n", " european elected \n"),
                 data["title"][1]. replace(" \n", " panther grizzly moose \n"),
                 data["title"][2]. replace(" \n", " lax dfw \n"),
                 data["title"][3]. replace(" \n", " geographic species \n"),
                 data["title"][4]. replace(" \n", " integrated connection alliance \n"),
                 data["title"][5]. replace(" \n", " trafficking convicted \n"),
                 data["title"][6]. replace(" \n", " railroad rail terminal \n"),
                 data["title"][7]. replace(" \n", " anti \n"),
                 data["title"][8]. replace(" \n", " renewable efficiency \n"),
                 data["title"][9]. replace(" \n", " european usdollar \n"),
                 data["title"][10]. replace(" \n", " victim admitting \n"),
                 data["title"][11]. replace(" \n", " vehicle manufacturer \n"),
                 data["title"][12]. replace(" \n", " trilogy mist rainforest \n"),
                 data["title"][13]. replace(" \n", " cristobol thunderstorms \n"),
                 data["title"][14]. replace(" \n", " imports chile produce \n"),
                 data["title"][15]. replace(" \n", " sword sculpture plate \n"),
                 data["title"][16]. replace(" \n", " regarding prohibiting violations \n"),
                 data["title"][17]. replace(" \n", " lense protect glare \n"),
                 data["title"][18]. replace(" \n", " religion christianity philosopher \n"),
                 data["title"][19]. replace(" \n", " scientist experiments innovated \n")
                 ]
print(expanded_list_in_out)

expanded_list_in_in = [data["title"][0]. replace(" \n", " men females \n"),
                 data["title"][1]. replace(" \n", " white red big \n"),
                 data["title"][2]. replace(" \n", " bwi transportation \n"),
                 data["title"][3]. replace(" \n", " wetlands conservation \n"),
                 data["title"][4]. replace(" \n", " system technology peripherals \n"),
                 data["title"][5]. replace(" \n", " trafficking illicit \n"),
                 data["title"][6]. replace(" \n", " polution routes highways \n"),
                 data["title"][7]. replace(" \n", " cybercrime \n"),
                 data["title"][8]. replace(" \n", " nitrogen electricity \n"),
                 data["title"][9]. replace(" \n", " european imperialist \n"),
                 data["title"][10]. replace(" \n", " raping cursing \n"),
                 data["title"][11]. replace(" \n", " vehicle car \n"),
                 data["title"][12]. replace(" \n", " fire snow wind \n"),
                 data["title"][13]. replace(" \n", " cristobol hurricane \n"),
                 data["title"][14]. replace(" \n", " tariffs countries imported \n"),
                 data["title"][15]. replace(" \n", " collectors engraved artwork \n"),
                 data["title"][16]. replace(" \n", " legal violations officers \n"),
                 data["title"][17]. replace(" \n", " sunlight skin glare \n"),
                 data["title"][18]. replace(" \n", " existentialism aristotle polytheism \n"),
                 data["title"][19]. replace(" \n", " scientist experiments advancements \n")
                 ]
print(expanded_list_in_in)


['\nWomen in Parliaments european elected \n', '\nBlack Bear Attacks panther grizzly moose \n', '\nAirport Security lax dfw \n', '\nWildlife Extinction geographic species \n', '\nHealth and Computer Terminals integrated connection alliance \n', '\nhuman smuggling trafficking convicted \n', '\ntransportation tunnel disasters railroad rail terminal \n', '\npiracy anti \n', '\nhydrogen energy renewable efficiency \n', '\neuro opposition european usdollar \n', '\nmercy killing victim admitting \n', '\nautomobile recalls vehicle manufacturer \n', '\nAmazon rain forest trilogy mist rainforest \n', '\ntropical storms cristobol thunderstorms \n', '\nCuba, sugar, exports imports chile produce \n', '\nart, stolen, forged sword sculpture plate \n', '\nlaw enforcement, dogs regarding prohibiting violations \n', '\nUV damage, eyes lense protect glare \n', '\nGreek, philosophy, stoicism religion christianity philosopher \n', '\ninventions, scientific discoveries scientist experiments innovated \n']


In [0]:
expanded_queries = data[['num','title']][:20]
expanded_queries['in_out_q'] = expanded_list_in_out
expanded_queries['in_in_q'] = expanded_list_in_in

In [0]:
expanded_queries.head()
filepath_expand = "/content/drive/My Drive/WashingtonPost.v2/data/expanded_queries.csv"
#expanded_queries.to_csv(filepath_expand)

